In [ ]:
%load_ext autoreload
%autoreload 2

!pip install pandas

import random
import pandas as pd

import helper.Counters as Counters
import helper.Inputs as Inputs
import helper.util as util
import helper.Powersort as Powersort
import helper.Timsort as Timsort

import helper.Books as Books

## Helper Functions

In [ ]:
def cost(lst, sorter):
    wrapped = [Counters.ComparisonCounter(x) for x in lst]
    Counters.reset_counters()
    sorter.sort(wrapped)
    assert Counters.ComparisonCounter.EQ_COMPARISONS == 0
    
    return {
        'Algorithm': sorter.name(),
        'Comparisons': Counters.ComparisonCounter.COMPARISONS
    }

In [ ]:
def compare_sorters(lst, sorters):
    sorters = sorted(sorters, key = lambda sorter: sorter.name())
    
    df = pd.DataFrame([cost(lst, sorter) for sorter in sorters])

    percent_improvement = ((df['Comparisons'][1] - df['Comparisons'][0]) / df['Comparisons'][1]) * 100

    
    return df, percent_improvement

## Random Permutation

The simplest way of generating an input list, without much thought, for the competition is to generate a set of random permutation.

In [ ]:
X = Inputs.random_permutation(10000)
name = "RandomPermutation"

compare_sorters(X, [Powersort, Timsort])

## Bible.txt


A more intuitive way is to base your input list on some real-world data. For this example, the list consists of words from the bible. Note that this is valid as alphabetical letters have inherent lexicographic ordering.

In [ ]:
X = Books.list_of_words_bible()
name = "Bible"

compare_sorters(X, [Powersort, Timsort])

## Mixed long and short runs

In [ ]:
def input_generator(n, rng):
    p = 0.8
    short = (1, 100)
    long = (1000, 10000)
    num_runs = max(2, int(n / (p * 0.5 * sum(short) + (1-p) * 0.5 * sum(long))))
    run_lens = Inputs.tims_random_run_lengths(num_runs, p, short, long, rng)
    nn = sum(run_lens)
    lst = [0] * nn
    
    Inputs.fill_with_asc_runs_same(lst, run_lens, 1, use_n_as_last_entry=False)
    lst = util.rank_reduce_ties_desc(lst)
    
    return lst

In [ ]:
X = input_generator(100000, random.Random(123))
name = "MixedLongShort"

compare_sorters(X, [Powersort, Timsort])

## Random unary strings

In [ ]:
def input_generator(n, rng):
    u = int(n ** 0.5)
    
    return Inputs.random_uary_array(u, n, rng)

In [ ]:
X = input_generator(1000000, random.Random())
name = "RandomUnary"

compare_sorters(X, [Powersort, Timsort])

## Extra (Helper) Function: save most recently generated list to filesystem

In [ ]:
print("Saving array as", name)

with open(name, "w") as f:
    f.write(str(X))
    f.close()